In this notebook, we are going to explore ways that one can apply Discrete Event Simulation when making decisions for hospital resource planning. The notebook is divided into multiple scenarios, each incorporating more uncertainties than the previous one. We encourage you to create your own Jupytor notebook and follow along. **If you do not have Python or Jupyter Notebook installed yet, you could also experiment with the virtual notebook by clicking "Launch Virtual Machine" (powered by Binder) above.** On the other hand, the dowloadable Jupyter Notebook can be found [here]().

# Background

---

Discrete Event Simulation (DES) is a useful decision making tool that can be applied to situations where resources are limited and queues exists in the process or system. Using DES, stakeholders can assess the outcomes, measured by specified Key Point Indicators (KPIs), of various scenarios (e.g. different bed capacities or staffing schedules in a clinic) without having to conduct experiments onsite (read more about DES itself [here](https://www.sciencedirect.com/topics/engineering/discrete-event-simulation)). Nowadays, there are many softwares that enable not only simulations but also animations of systems and processes. However, in this notebook, we will take a look at a process-based discrete event simulation framework within Python called SimPy. Through this notebook, we hope to give you some basic ideas of what some of the software packages are doing behind the scene.

Before we begin, please download and install the following packages (if you are using the virtual environment through the link above, these packages would already be installed):

In [27]:
import simpy as sim
import numpy as np
import pandas as pd

In the package SimPy, simulations are conducted within an `environment`. Some tutorials initiate the `environment` after all the resources and processes are defined, but we have found it more intuitive to keep it at the beginning so that people know what the variable `env` refers to later on in different Python `functions` or `classes`.

In [28]:
env = sim.Environment()

# Scenario One: Single Resource (Bed)

---

In the first scenario, we want to start by building a really simple (though unrealistic) simulation model with minimum uncertainties. Let's assume the following for a walk-in clinic:

- There is only one required resource for each patient: a bed
- There is only one bed in the clinic
- There is no need for doctors or nurses (we will build them into the simulation later)
- The clinic opens from 9am to 5pm
- The first patient arrives at exactly 9am
- After that, a new patient arrives every half an hour
- THe last patient walks in at excatly 4:30pm
- Each patient uses the bed for exactly 30 minutes (for diagnosis)

First, as you may have already imagined, we will set up the patient arrival schedule based on their interarrival time (which later on would be generated using random variables). We will also assign the diagnosis time to a variable so that our code could be more dynamic for the future:

In [29]:
arr_sche = np.arange(9.0, 17.0, 0.5) # you could also try np.linspace
num_patients = len(arr_sche)
diag_time = 0.5

print(arr_sche)

[ 9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5 14.  14.5 15.  15.5
 16.  16.5]


After you have created the arrival schedule and diagnosis time, we can go back to the `environment` that we initiated and create withint the `environment` our `resource`: bed

In [30]:
bed = simpy.Resource(env, capacity=1)

In more complicated discrete simulation models, one would usually need to utilize Python `class` (read more about Python `class` [here](https://docs.python.org/3/tutorial/classes.html)). However, since we are starting simple, we can just define a `function` first, and create `classes` later when we move into scenarios with more resources and uncertainties. 

In the function that we define below, we want the user to pass in five arguments: the `environment` of the simulation, the patient ID (which will actually be defined later when we run the simulation, the resource needed, the arrival time of the patient (taken from our schedule), and the length that this resource is going to be used (diagnosis time). The function contains `yield` instead of `return`, which allows it to be come a `generator` (read more about Python `generators` [here](https://wiki.python.org/moin/Generators). Take a look at the function first, and we will explain it after the cell.

In [31]:
def pat(env, pat_id, resource, arr_time, diag_time):
    
    yield env.timeout(arr_time)
    print("{} arrives at the clinic at {} o'clock".format(pat_id, env.now))
    
    with resource.request() as req:
        
        yield req
        print("{} starts using the bed at {} o'clock".format(pat_id, env.now))
        
        yield env.timeout(diag_time)
        print("{} leaves the clinic at {} o'clock".format(pat_id, env.now))

In SimPy, simulation normally starts at time 0 unless specified. Time is also unit less within SimPy, so we can define it however we want, though we do need to keep it consistent. In the function above, we first let the simulation run with no events until the time when the first patient arrives. Then we generate the first patient, who will request for the resource, our one bed. If the resource is available (it will always be available because of our current setup), the patient takes over the resource until it finishes using it; however, if the resource is not available, the patient will wait in a queue and once the resource becomes available it will be released to the next patient in line. 

After we set up the function that we need for the simulation, we then have to create `processes` for each of the patient using the `generator` that created earlier (read about why `entities` are considered `processes` in SimPy [here](https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html)):

In [32]:
for i in range(num_patients):
    
    env.process(pat(env, 'Patient {}'.format(i), bed, arr_sche[i], diag_time))

Finally, we can run our simulation, and you will see a detailed description of all the events under the cell below:

In [33]:
env.run()

Patient 0 arrives at the clinic at 9.0 o'clock
Patient 0 starts using the bed at 9.0 o'clock
Patient 1 arrives at the clinic at 9.5 o'clock
Patient 0 leaves the clinic at 9.5 o'clock
Patient 1 starts using the bed at 9.5 o'clock
Patient 2 arrives at the clinic at 10.0 o'clock
Patient 1 leaves the clinic at 10.0 o'clock
Patient 2 starts using the bed at 10.0 o'clock
Patient 3 arrives at the clinic at 10.5 o'clock
Patient 2 leaves the clinic at 10.5 o'clock
Patient 3 starts using the bed at 10.5 o'clock
Patient 4 arrives at the clinic at 11.0 o'clock
Patient 3 leaves the clinic at 11.0 o'clock
Patient 4 starts using the bed at 11.0 o'clock
Patient 5 arrives at the clinic at 11.5 o'clock
Patient 4 leaves the clinic at 11.5 o'clock
Patient 5 starts using the bed at 11.5 o'clock
Patient 6 arrives at the clinic at 12.0 o'clock
Patient 5 leaves the clinic at 12.0 o'clock
Patient 6 starts using the bed at 12.0 o'clock
Patient 7 arrives at the clinic at 12.5 o'clock
Patient 6 leaves the clinic 